In [50]:
import pandas as pd
import json
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

file = 'data/reddit/controversial-comments.jsonl'

data = []

with open(file) as f:
    for line in f:
        data.append(json.loads(line))
        
data=data[:100000]

In [51]:
stop_words = stopwords.words('english')

In [52]:
#def pre_process(text):
    #text=text.lower()
    #text=re.sub('&lt;/?.*?&gt;',' &lt;&gt', text)
    #text=re.sub('\\d|\\W+',' ',text)
    #return text

In [63]:
#df = pd.DataFrame(data)
#df['txt']=df['txt'].apply(lambda x:pre_process(x))
#df.head()

In [60]:
#docs=df['txt'].tolist()
#cv=CountVectorizer(max_df=0.85, stop_words=stop_words)
#wcv=cv.fit_transform(docs)

dv = DictVectorizer()
features=vec.fit_transform(data[:1000]).toarray()
feature_names=vec.get_feature_names()
pd.DataFrame(features, columns=feature_names)

,con,"txt=""Binders full of women"" was about his disconnect and superficial efforts. The others weren't exactly headline news. Of course Jezebel is going to push that crap. I'm talking about serious journalists. Not shitstains","txt=""He's bigly dumb.""","txt=""Hillary Clinton, who has confessed to sharing classified intel with boy toy in the past considered for SOS in Obamas cabinet."" I wonder how that headline would be received by the right.","txt=""ISIS doing their bloodletting here"" what the fuck","txt=""Lol"" -DEA","txt=""MUH FASCISM!!!!!!!"" Fear mongering is all you libs can do now that you don't get your way any time you want, huh? I can't wait until you cry when gay marriage gets repealed.","txt=""Quote the Constitution and those questioning me will be called unpatriotic.""","txt=""Reeee"" ye totally","txt=""Should"" No.",...,"txt=um, he literally bragged about it...",txt=well aren't **you** being overly dramatic. /s,txt=what a crock,txt=what history? you mean the completely unproven claims? moving on,txt=what kind of asshole goes to a bar where they show cable news?,txt=what? http://www.nytimes.com/2016/11/26/us/politics/clinton-camp-will-join-push-for-wisconsin-ballot-recount.html nice try,"txt=yep GW passed off the hard decision to Obama, hot potato.","txt=you are forgetting your ears... you would still hear Fox news, even if you could not see it.",txt=you support conservatives... who hate peoples rights.,"txt=“He is considering splitting his time between the White house and his Manhattan residence, which would cost tax-payers tens of millions of dollars. But, it’s all worth it to help a billionaire go night-night in his big boy bed.” (from SNL Weekend Update)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
target=list(range(1000))
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.1, random_state=1)

In [79]:
standardizer = StandardScaler()
standardizer.fit(features_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [80]:
features_train_std = standardizer.transform(features_train)
features_test_std = standardizer.transform(features_test)